In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj0.ipynb")

# Project 0 - SQL Review -- Music Querying
## Due Date: Wednesday, September 10th, 5:00pm PST

In this project, we will be working with SQL on a Billboard Music database.

## Collaboration Policy
Please read out [course collaboration policy](https://data101.org/fa25/syllabus/#collaboration-and-integrity). Do not share code. If you discuss the assignments with others, please include their names below.

**Collaborators**: _list collaborators here_


## Objectives

- Review SQL syntax from prerequisite courses

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel, clear output, and simply re-run the notebook as normal. This will shutdown your current connection to the database, which will prevent the issue of multiple connections to the database at any given point. When re-running the notebook, you will create a fresh database based on the provided Postgres dump.

If you face slow kernel times or are unable to open your notebook, restart your server by going to File -> Hub Control Panel -> Stop My Server and then clicking Start My Server.

## Logistics & Scoring Breakdown

Please read the submission instructions carefully and double check that your submission is not throwing any errors. Please ensure that public tests pass upon submission. It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests. If there are free-response questions (marked 'm' in the table below), they are manually graded.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

|Question|Points|
|---|---|
|0|3|
|1|3|
|2|3|
|3|3|
|4|3|
|5a|3|
|5b|3|
|6|0|
|**Total**|21|

**Summary:** 21 points (autograded: 21, manual: 0) 

In [ ]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Before You Start

## Assignment Tips
We recommend you read through <a href="https://data101.org/fa25/assignment-tips/">Fall 2025 Assignment Tips</a>—a handy resource that has many other tips that we **highly recommend** taking a moment to read through to save you time and improve your workflow.

* PostgreSQL documentation
* Jupyter cell magic
* JupySQL and magic commands in Jupyter
* JupyterHub keyboard shortcuts
* psql and common meta-commands (not required for Proj 0)
* Debugging:
    * Where to create new cells to play nice with the autograder
    * Opening/closing connections, deleting databases if all else fails
* Local installation (not supported by staff officially, but for your reference)

## Table of Contents
Jump to different parts of this notebook:
* [Before You Start](#Before-You-Start): This section
* [Setup](#Setup): Database Setup, Grader Setup (make sure to run these cells)
* [The `billboard` Database](#The-billboard-Database): Information about this project's database schema, plus optional features for you to explore
* [Question 0](#q0): The first assignment question


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Setup

## Database Setup
We are going to be using the `JupySQL` library to connect our notebook to a PostgreSQL database server on your JupyterHub account. Running the next cell will do so; you should not see any error messages after it executes.

In [ ]:
# The first time you are running this cell, you may need to run the following line as: %load_ext sql
%reload_ext sql

In the next cell, we will unzip the data. This only needs to be done once.

In [ ]:
!unzip -uo data/billboard.zip -d data/

<br/>

**Create the `billboard` database**: <br>
We will use PostgreSQL commands to create a database and import our data into it. Run the following cell to do this. It may take a few seconds.

* You can also run these cells in the command-line via `psql`.
* If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [ ]:
!psql postgresql://localhost/billboard -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql postgresql://localhost/postgres -c 'DROP DATABASE IF EXISTS billboard'
!psql postgresql://localhost/postgres -c 'CREATE DATABASE billboard'
!psql postgresql://localhost/billboard -f data/billboard.sql -q

**Connect to `billboard` database in the Notebook**: 
<br>
Now let's connect to the new database we just created! There should be no errors after running the following cell.

In [ ]:
%sql postgresql://localhost/billboard

Assuming the previous command doesn't show an error, this will use the current connection to show all tables in the `billboard` database.

In [ ]:
%sqlcmd tables

---

**Quick check**: To make sure things are working, let's fetch 10 rows from one of our tables `tiktok_top_50`. Just run the following cells. If there are no errors, no further action is needed.

In [ ]:
%%sql
SELECT * FROM tiktok_top_50 LIMIT 10;

## Grader Setup

In [ ]:
# Connecting the grader
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj0")
grading_util.prepare_autograder()

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# The `billboard` Database

[Billboard](https://www.billboard.com) is a music industry publication which tracks the sales and distribution of music across many genres. Each week, they publish “charts” which aggregate the total sales, streams (e.g. via Spotify, Apple Music, or YouTube) as well as radio airplay in the US.
 
The most (in)famous chart is the [Billboard Hot 100](https://www.billboard.com/charts/hot-100/), widely regarded as the list of most popular songs in the US. For this assignment, you’ll be looking at data from the Hot 100 list, primarily in recent years — though the database we’ve provided to you goes all the way back to 1958! 

Later questions will involve `JOIN`ing with data from the [Billboard TikTok Top 50](https://www.billboard.com/charts/tiktok-billboard-top-50/), a list of the songs that are the most popular on TikTok, factoring in views and user engagement.

Below is a list of the relations (tables) in our database. 
- **hot_100**: Metadata for songs in the Billboard Hot 100
- **tiktok_top_50**: Metadata for songs in the Billboard TikTok Top 50

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

## Data Schema

When approaching a new database, one of the most important things to do is understand the database schema. Remember that a database is a set of tables, which each contain their own schema. For this assignment, both the `hot_100` and `tiktok_top_50` tables contain the following schema.

```
CREATE TABLE hot_100 (
    week_ending TEXT,
    rank INTEGER,
    title TEXT,
    artist TEXT, 
    image_url TEXT,
    peak_position INTEGER,
    weeks_on_chart INTEGER,
    UNIQUE (week_ending, rank)
);
```

- `week_ending`: The week that "chart" was published, as a `YYYY-MM-DD` string

- `rank`: The track's current position on the chart. (A rank of 1 is better than a rank of 10.)

- `title`: Track's title

- `artist`: Track's artist

- `image_url`: Image URL of the track's album cover

- `peak_position`: The track's peak position on the chart as of the chart date

- `weeks_on_chart`: The number of weeks the track has been or was on the chart up until that point

**Note:** In the case of Billboard charts, a song (or album's) `peak_position` is based on the idea that being #1 is the 'best'. Therefore, a _smaller_ value for `peak_position` is better. 

_An aside_: ⏰ We've decided to use a `TEXT` data type for the `week_ending` date. Later we'll explore the `date` type. However, we can perform ordered comparisions on these strings, since `'2024-01-01'` comes before `'2024-01-02'` lexicographically. 

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

## Optional Fun

Call the `display_query` function passing in your SQL query result to view the results with a table that links to Apple Music and Spotify. An example usage of the `display_query` function can be found at the end of Question 0.

In [ ]:
# Some utilities to make exploration easier.
# NOTE: Make sure to write display_query(...);
# Use the ; to supress the notebook's default output.

# Common Configuration
DISPLAY_ALBUM_ART = False
IMAGE_SIZE = '50px' # Adjust to change the size of album art.
MAX_ROWS = 250

import urllib.parse
from IPython.display import display, HTML

# Adjust pandas display options to better handle large DataFrames
# Note: These settings do not apply when calling `df.to_html`
pd.set_option('display.max_rows', 500) # Adjust as needed
pd.set_option('display.max_seq_items', 250)
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.colheader_justify', 'left')

spotify_url = 'https://play.spotify.com/search/'
apple_music_url = 'https://music.apple.com/us/search?term='

# You probably don't need this function but you can use it make "fancy"
# datatables which are sortable and searchable. (But then can be a little slow to load.)
def html_datatable(html):
    return display(HTML(f"""
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.21/css/jquery.dataTables.css">
    <script type="text/javascript" charset="utf8" src="https://code.jquery.com/jquery-3.5.1.js"></script>
    <script type="text/javascript" charset="utf8" src="https://cdn.datatables.net/1.10.21/js/jquery.dataTables.js"></script>
    <script>
    $(document).ready(function() {{
        $('table.display').DataTable();
    }});
    </script>
    <div>{html}</div>
    """))

def basic_table(html):
    return display(HTML(f"""
    <div style="height: 300px; overflow-y: scroll; border: 1px solid #ccc;">
        {html}
    </div>
    """))

def display_query(result, show_search=True):
    """
    A fairly basic df to HTML-table display utilitiy.
    Includes the ability to give links to Spotify/Apple Music for easy exploring and maximal time wastage.
    """
    song_query = lambda row: urllib.parse.quote(row.get("title", '') + " " + row.get("artist", ''))
    format_link = lambda col_name: lambda cell: f'<a href="{cell}" target="_blank">{col_name}</a>'
    formatters = {
        'image_url': lambda image_url: f'<img src="{image_url}" alt="Song Cover Art" style="height: {IMAGE_SIZE}; width: auto; max-height: 100%;">',
        'spotify': format_link('Spotify'),
        'apple': format_link('Apple Music'),
    }

    # Display the DataFrame as a nicely formatted table with scroll bars
    df = result.DataFrame()
    if show_search and ('artist' in df.columns or 'title' in df.columns):
        df['spotify'] = df.apply(lambda row: f'{spotify_url}{song_query(row)}', axis=1)
        df['apple'] = df.apply(lambda row: f'{apple_music_url}{song_query(row)}', axis=1)

    if not DISPLAY_ALBUM_ART and 'image_url' in df.columns:
        df = df.drop(columns='image_url')

    # Convert DataFrame to HTML
    # We must manually set out own limits!
    html_table = df.to_html(index=True, escape=False, formatters=formatters, show_dimensions=True, max_rows=MAX_ROWS)

    # Swap these two lines to see the fancy version of the output.
    # html_datatable(html_table)
    basic_table(html_table)
    return df;

<br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

<a name="q0"></a>

# Question 0 — Warmup


This question is given to you! Verify that your table loads correctly.
Run each of the following queries, and you should see the right output.

In [ ]:
%%sql --save query_0_0 result_0_0 <<
SELECT COUNT(*) FROM hot_100

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0_0 = %sqlcmd snippets query_0_0
grading_util.save_results("result_0_0", query_0_0, result_0_0)
result_0_0

In [ ]:
%%sql --save query_0_1 result_0_1 <<
SELECT * FROM hot_100 LIMIT 25

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0_1 = %sqlcmd snippets query_0_1
grading_util.save_results("result_0_1", query_0_1, result_0_1)
result_0_1

In [ ]:
%%sql --save query_0_2 result_0_2 <<
SELECT * FROM hot_100 ORDER BY week_ending DESC LIMIT 30

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0_2 = %sqlcmd snippets query_0_2
grading_util.save_results("result_0_2", query_0_2, result_0_2)
result_0_2

In [ ]:
display_query(result_0_2);

In [ ]:
grader.check("q0")

<br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 1 — Recent Hits


Select all song titles and artists on the Billboard Hot 100 chart in March 2024 which have ranking of 15 or better, sorted alphabetically by artist name.

Notes:
- The column `week_ending` is a **string**. it is represented `YYYY-MM-DD`. Think back to string comparisons in Data 100.  (Later in this class we will explore Postgres date types.)

- Rank 1 is the best, so make sure you are using the right comparison operator!

- Remember that a **unique** pair of song and artist can appear multiple times on the chart (for example, if the song charts for multiple weeks). Make sure your output only includes each song once.

In [ ]:
%%sql --save query_1 result_1 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1 = %sqlcmd snippets query_1
grading_util.save_results("result_1", query_1, result_1)
result_1

In [ ]:
display_query(result_1);

In [ ]:
grader.check("q1")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />

# Question 2 — Longest Time Spent on the Charts


Find all song titles, their artist, and the total number of weeks the song spent on the Billboard Hot 100 (call this column `total_weeks_on_chart`).

Only include songs that have spent >8 weeks on the chart.

Sort by the total number of weeks the song spent on the chart, descending order.

Notes:
- Similar to the previous question, make sure each song appears once
- The `weeks_on_chart` column is the number of weeks on the chart **as of that week**. How can we get the total number of weeks?
- Do **NOT** count the total number of rows in which that the song/artist pair appear on the hot_100 table

In [ ]:
%%sql --save query_2 result_2 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2 = %sqlcmd snippets query_2
grading_util.save_results("result_2", query_2, result_2)
result_2

In [ ]:
display_query(result_2);

In [ ]:
grader.check("q2")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 3 — Longest Top 10 Hits of the 1970's


Find all song titles, their corresponding artist, and its (highest) peak position (call this column `highest_peak_position`).

Only include songs...
- with a highest peak position in the top 10 (e.g. position 1-10, inclusive) AND
- that charted from Jan. 1, 1970 - Dec. 31 1979, inclusive

Sort by `highest_peak_position` in ascending order **AND** the song title in ascending order to break tie.

Notes:
- Remember "highest" position refers to lower numbers (e.g. position of 5 is higher than position of 10)
- Peak position refers to the historical highest rank of a song, while rank only indicates the rank of that specific week

In [ ]:
%%sql --save query_3 result_3 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_3 = %sqlcmd snippets query_3
grading_util.save_results("result_3", query_3, result_3)
result_3

In [ ]:
display_query(result_3);

In [ ]:
grader.check("q3")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 4 — Top Artists By Number 1 Hits


Find the top 50 _artists_ by total number of weeks any of their songs spent at rank 1 (call this column `total_weeks_at_number_one`).

Sort by `total_weeks_at_number_one` in descending order **AND** the name of the artist in ascending order to break tie.

Example: Suppose artist Jane Doe has 3 songs A, B, and C that have spent 10, 20, and 30 weeks at rank 1, respectively. Then the expected output would be: Jane Doe, 60 because 10 + 20 + 30 = 60 weeks total.

Notes:
- Remember that an artist can have multiple songs that have reached rank 1 at some point
- Remember that every row in the `hot_100` table represents 1 week on the chart, so be careful about how you aggregate the rows. Which aggregation function would work best here?

In [ ]:
%%sql --save query_4 result_4 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4 = %sqlcmd snippets query_4
grading_util.save_results("result_4", query_4, result_4)
result_4

In [ ]:
display_query(result_4);

In [ ]:
grader.check("q4")

<br/><br/><br/>

<hr style="border: 1px solid #fdb515;" />


# Question 5 — What's Trending on TikTok?

## Question 5a — Popular Everywhere

Find the songs and their corresponding artists that were on both the Billboard Hot 100 and TikTok Top 50 charts in the same week.

Sort in ascending order by song title.

Notes:
- Remember that the same song can appear in multiple rows of the chart (for example, if the song was on the chart for multiple weeks)
- A song is considered the "same" as another song if the song title and artist match

In [ ]:
%%sql --save query_5_0 result_5_0 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5_0 = %sqlcmd snippets query_5_0
grading_util.save_results("result_5_0", query_5_0, result_5_0)
result_5_0

In [ ]:
display_query(result_5_0);

In [ ]:
grader.check("q5a")

## Question 5b — Brat Summer

Write a query that ouputs:

- `week_ending`
- Billboard Hot 100 peak position for that week (call this column `hot_100_peak_pos`)
- TikTok Top 50 peak position for that week (call this column `tiktok_50_peak_pos`)
- Song title

For all songs where the artist is `'Charli xcx'` (case sensitive). 

Sort by `week_ending`, ascending order.

*Ensure that only we only consider songs appearing on the Billboard Hot 100 and TikTok Top 50!*

<div>
<img src="https://i.kym-cdn.com/photos/images/newsfeed/002/862/673/d07.jpg" alt="Brat summer starter pack meme" width="300"/>
</div>

In [ ]:
%%sql --save query_5_1 result_5_1 <<
...

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5_1 = %sqlcmd snippets query_5_1
grading_util.save_results("result_5_1", query_5_1, result_5_1)
result_5_1

In [ ]:
display_query(result_5_1);

In [ ]:
grader.check("q5b")

<br/><br/>

<hr style="border: 1px solid #fdb515;" />

# Question 6 — Reminders!

Fill out the [pre-semester form](https://docs.google.com/forms/d/e/1FAIpQLScUTyUXPdLlgbFEWQJGYWojCg5dZJ3V55Qo18CyH0J2UfX3Gg/viewform). Then in the code cell below, set `secret_word` to the secret word at the end of the form.

In [ ]:
secret_word = ...

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Congratulations! You have finished Project 0.

We hope you found something interesting to listen to :D

Here is an optional but interesting video by Vox related to viral TikTok songs: [We tracked what happens after TikTok songs go viral](https://youtu.be/S1m-KgEpoow?feature=shared)

#### Final Question

What's something fun/weird/interesting you discovered while exploring some data in this assignment? (Totally feel free to leave this blank...)

> ...put you answer in this cell.

## Acknowledgments

This assignment was inspired by [Chris Molanphy’s podcast “Hit Parade”](https://slate.com/podcasts/hit-parade) which dives into the history of popular music in the US. Former TA Allen Guo’s [billboard.py](https://github.com/guoguo12/billboard-charts) library made it possible to easily extract the data from Billboard’s ad-riddled website. 

If you’d like to explore any of the other charts, or perhaps query more recent data, you should give it a try.

## Submission

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For your submission on Gradescope, you will need to submit the `proj0.zip` file generated by the export cell.** Please ensure that your submission includes `proj0.ipynb`, and `results.zip`. 

**Please ensure that public tests pass upon submission.** It is your responsibility to wait until the autograder finishes running. We will not be accepting regrade requests for submission issues.

**Common submission issues:** You MUST submit the generated zip file to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 0 directory within JupyterHub (remove `proj0.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

In [ ]:
grading_util.prepare_submission_and_cleanup()

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, files=['results.zip'])